<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/xerces-java-trunk/xerces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

save_file_csv = 'xerces2.csv'
file_csv =  'xerces.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/xerces-java-trunk'

In [4]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [5]:
file_name(file_dir)

In [6]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}


In [7]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line) #Exclude Chinese characters
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese) #Exclude Chinese characters
    #print("unchinese:",unchinese)
    return unchinese

In [8]:
def del_stopwords(line):

    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

    return new_line

In [9]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
            if len(coefs) != 0:
                if coefs != ' ':
                    if coefs != '\n':
                        for i in coefs:
                            list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)

T
T
T
T
T
T
T
T
T
T
T
T
T
T
T


In [10]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['DeferredElementImpl', 'False'],
       ['DVFactoryException', 'False'],
       ['XSImplementation', 'False'],
       ...,
       ['XSException', 'False'],
       ['DOMInputSource', 'False'],
       ['DOMInputImpl', 'False']], dtype='<U37')

In [11]:
len(index)

638

In [12]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['DeferredElementImpl', 'This', 'is', 'the', 'deferred', 'constructor', 'Only', 'the', 'fNodeIndex', 'is', 'given', 'here', 'All', 'other', 'data', 'can', 'be', 'requested', 'from', 'the', 'ownerDocument', 'via', 'the', 'ownerDocint'])
 list(['DVFactoryException', 'DVFactoryExceptionString'])
 list(['XSImplementation', 'Creates', 'an', 'immutable', 'codeLSInputListcode', 'from', 'the', 'given', 'array', 'of', 'param', 'values', 'the', 'array', 'containing', 'the', 'codeLSInputcode', 'values', 'that', 'will', 'be', 'placed', 'in', 'the', 'return', 'an', 'immutable', 'codeLSInputListcode', 'from', 'the', 'given', 'array', 'of', 'LSInputList', 'createLSInputListLSInput', 'Creates', 'an', 'immutable', 'codeStringListcode', 'from', 'the', 'given', 'array', 'of', 'param', 'values', 'the', 'array', 'containing', 'the', 'codeStringcode', 'values', 'that', 'will', 'be', 'placed', 'in', 'the', 'return', 'an', 'immutable', 'codeStringListcode', 'from', 'the', 'given', 'array', 'of', 'String

In [13]:
# #training phase
def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
    # print(vectorizer.get_feature_names())#Get the vocabulary found by the model directly analyzing the data（collection of words above）
    # print(X.toarray())# Printing X directly outputs the position of each word
    #     print(X)

    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)  # Latent semantic analysis, set 4 topics
    X2 = svd_model.fit_transform(X)  # train and convert

    # print("--------lsa Singular Value---------")
    # print(svd_model.singular_values_)
    # print("--------Representation of text in topic vector space Left singular matrix---------")
    # print(X2)

    terms = vectorizer.get_feature_names()
    # print("-------- Topics and keywords ---------")
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

In [14]:
len(index[0:200])

200

In [15]:
q = 0
acsm= []
for i in index[0:200]:
    lists = []
    print("*************",i,"****************")
    # print("------------------------------------------------------------------")
    for j in i:
        lists.append(j)
    # lists = deleteDuplicatedElementFromList(lists)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)

************* ['DeferredElementImpl', 'This', 'is', 'the', 'deferred', 'constructor', 'Only', 'the', 'fNodeIndex', 'is', 'given', 'here', 'All', 'other', 'data', 'can', 'be', 'requested', 'from', 'the', 'ownerDocument', 'via', 'the', 'ownerDocint'] ****************
the
other
here
requested
deferred
ownerdocint
fnodeindex
 
************* ['DVFactoryException', 'DVFactoryExceptionString'] ****************
dvfactoryexception
dvfactoryexceptionstring
 
************* ['XSImplementation', 'Creates', 'an', 'immutable', 'codeLSInputListcode', 'from', 'the', 'given', 'array', 'of', 'param', 'values', 'the', 'array', 'containing', 'the', 'codeLSInputcode', 'values', 'that', 'will', 'be', 'placed', 'in', 'the', 'return', 'an', 'immutable', 'codeLSInputListcode', 'from', 'the', 'given', 'array', 'of', 'LSInputList', 'createLSInputListLSInput', 'Creates', 'an', 'immutable', 'codeStringListcode', 'from', 'the', 'given', 'array', 'of', 'param', 'values', 'the', 'array', 'containing', 'the', 'codeStri

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


************* ['XSParticle', 'max', 'occurs', 'determines', 'the', 'maximum', 'number', 'of', 'terms', 'that', 'can', 'occur', 'To', 'query', 'for', 'the', 'value', 'of', 'unbounded', 'use', 'codemaxOccursUnboundedcode', 'When', 'the', 'value', 'of', 'codemaxOccursUnboundedcode', 'is', 'codetruecode', 'the', 'value', 'of', 'codemaxOccurscode', 'is', 'unspecified', 'int', 'max', 'occurs', 'whether', 'the', 'maxOccurs', 'value', 'is', 'boolean', 'min', 'occurs', 'determines', 'the', 'minimum', 'number', 'of', 'terms', 'that', 'can', 'occur', 'int', 'term', 'one', 'of', 'a', 'model', 'group', 'a', 'wildcard', 'or', 'an', 'element', 'declaration', 'XSTerm'] ****************
of
int
can
number
max
determines
boolean
 
************* ['DOMImplementationListImpl', 'Construct', 'an', 'empty', 'list', 'of', 'fImplementationsnew', 'Construct', 'a', 'list', 'of', 'DOMImplementations', 'from', 'an', 'DOMImplementationListImplArrayList', 'Construct', 'a', 'list', 'of', 'DOMImplementations', 'from', '

In [16]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)

c = np.array(c)

In [17]:
from keras import Input,Model
from keras.layers import Dense

In [18]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list


In [19]:
def word2vec_train(index):
    list = []
    list.append(index[0])
    words = []

    additem(index)
    # for text in index:
    #     #     print(text)
    print(index)
    for word in index:
        if word not in words:
            words.append(word)

    # words = set(words)
    # print(words)

    word2int = {}
    for i,word in enumerate(words):
        word2int[word] = i
    print(word2int)



    WINDOW_SIZE=2
    data = []
    # for sentence in index:
    for idx,word in enumerate(index):
        for neighbor in index[max(idx - WINDOW_SIZE,0) : min(idx + WINDOW_SIZE,len(index))+ 1]:
            if neighbor != word:
                data.append([word, neighbor])
    #                 print(data)
    df = pd.DataFrame(data,columns=['input','label'])

    # print(df)

    ONE_HOT_DIM = len(words)
    #function to convert numbers to one hot vectors
    def to_one_hot_encoding(data_point_index):
        one_hot_encoding = np.zeros(ONE_HOT_DIM)
        one_hot_encoding[data_point_index] = 1
        return one_hot_encoding

    X = []
    Y = []

    for x, y in zip(df['input'],df['label']):
        X.append(to_one_hot_encoding(word2int[x]))
        Y.append(to_one_hot_encoding(word2int[y]))

    x_train = np.array(X)
    y_train = np.array(Y)


    #Defining the size of the embedding
    embed_size = 3
    xx = Input(shape=(x_train.shape[1],))
    yy = Dense(units=embed_size, activation='linear')(xx)
    yy =Dense(units=y_train.shape[1],activation='softmax')(yy)
    model = Model(inputs=xx, outputs=yy)
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')
    #optimizing the network weights
    model.fit(x=x_train,y=y_train,
            batch_size=16,
            epochs=10
    )
    weights = model.get_weights()[0]
    # embedding = []
    # embedding.append[index[0]]

    for word in words:
        # list = []
        # list.append(word)
        q = weights[df.get(word)]

        arr_new = np.sum(q, axis=1)
        print('-----------------------')
        print(arr_new.shape)
        print('-----------------------')
        arr_new = arr_new.tolist()

        list.append(arr_new)

        break
        # list.append(q)
        # # print(weights[df.get(word)].shape)
        # print(word)
        # print(q)
        # print('---')
        # embedding.append(list)
    return list

In [20]:
embedding = []
for item in index[0:200]:
    embedding.append(word2vec_train(item))
print(embedding)

Streaming output truncated to the last 5000 lines.
{'DeferredElementImpl': 0, 'This': 1, 'is': 2, 'the': 3, 'deferred': 4, 'constructor': 5, 'Only': 6, 'fNodeIndex': 7, 'given': 8, 'here': 9, 'All': 10, 'other': 11, 'data': 12, 'can': 13, 'be': 14, 'requested': 15, 'from': 16, 'ownerDocument': 17, 'via': 18, 'ownerDocint': 19}
Epoch 1/10
6/6 [==============================] - 1s 4ms/step - loss: 3.0352
Epoch 2/10
6/6 [==============================] - 0s 12ms/step - loss: 3.0315
Epoch 3/10
6/6 [==============================] - 0s 4ms/step - loss: 3.0281
Epoch 4/10
6/6 [==============================] - 0s 6ms/step - loss: 3.0245
Epoch 5/10
6/6 [==============================] - 0s 6ms/step - loss: 3.0216
Epoch 6/10
6/6 [==============================] - 0s 4ms/step - loss: 3.0184
Epoch 7/10
6/6 [==============================] - 0s 5ms/step - loss: 3.0154
Epoch 8/10
6/6 [==============================] - 0s 3ms/step - loss: 3.0124
Epoch 9/10
6/6 [==============================] - 0s 3

In [41]:
df =pd.read_csv("/content/drive/MyDrive/delesmell/dataset_class/xerces-java-trunk/xerces.csv") 

In [42]:
x= df.iloc[:, 0]

In [43]:
ngList = []

In [44]:
for index in x:
#     print(index)
    q = 1
    for i in range(0,len(embedding)):
        st = str(embedding[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            # embedding[i][1].reshape((1,3))
            # l = embedding[i][1].tolist()
            ngList.append(embedding[i][1])
            q = 0
            # break
#             print('*')
    if q==1 :
        ngList.append([[1, 1, 1]])



word2vec = []
list1 = []
list2 = []
list3 = []
for i in range(0,len(ngList)):
    word2vec.append(ngList[i][0])

for i in range(0,len(word2vec)):
    list1.append(word2vec[i][0])
for i in range(0,len(word2vec)):
    list2.append(word2vec[i][1])
for i in range(0,len(word2vec)):
    list3.append(word2vec[i][2])


In [45]:
for i in range(0,len(Brain_Class)):
    if 'False' == Brain_Class[i][1]:
        q = 1
    else :
        print('*')

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


In [46]:
#add new column
ngList=[] #Prepare a list and store the data of the new column in it

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(c[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            ngList.append(c[i][1])
            q = 0
#             print('*')
    if q==1 :
        ngList.append(1)

In [47]:
brainList=[] #Prepare a list and store the data of the new column in it

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(Brain_Class[i][0])
        st = st[1:]
        if index == st:
            brainList.append(Brain_Class[i][1])
            q = 0
    if q==1 :
        brainList.append('0')
        
        
print(len(brainList))
print(len(x))

1155
1155


In [48]:
column=df.columns.tolist()

In [49]:
df['acsm'] = ngList 
df['is_brain'] = brainList 
df.to_csv(save_file_csv,mode = 'a',index =False)

In [52]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOA,NOAM,NOD,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain
0,Assertion,2.50,0,0,1.0,0,0,0,0.0,0,...,0,0,0,4,0,0.17,10,1.0,1,0
1,Counter,5.15,6,0,1.0,9,0,0,3.5,0,...,22,0,0,13,0,0.29,67,1.0,1,0
2,DocumentTracer,2.85,11,0,1.0,13,0,0,4.0,2,...,17,0,0,60,0,0.83,171,1.0,1,0
3,1,1.00,0,0,1.0,0,0,0,0.0,0,...,0,0,0,1,0,1.00,1,1.0,1,0
4,2,1.00,0,0,1.0,0,0,0,0.0,0,...,0,0,0,1,0,1.00,1,1.0,1,0
